In [ ]:
import pandas as pd
import numpy as np
import datetime
import glob
import os

In [ ]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Readin merged data
house1 = pd.read_csv('~/Desktop/data-x-data/ProcessedData/House1/House1_Appliances.csv',index_col = 0, parse_dates = True)

In [ ]:
house1.head(1)

In [ ]:
# Get DayOfWeek & Month
DayOfWeek = house1.index.weekday
month = house1.index.month
house1['DayOfWeek'] = DayOfWeek
house1['Month'] = month

In [ ]:
# Define function to add Day Of Week to data in one-hot-encoding form
def DayOfWeek_one_hot_encoding(df):
    DayOfWeek = df.index.weekday
    df['DayOfWeek'] = DayOfWeek
    df = df.join(pd.get_dummies(df['DayOfWeek']))
    df.rename(columns={0:'Monday',\
                       1:'Tuesday',\
                       2:'Wednesday',\
                        3:'Thursday',\
                        4:'Friday',\
                        5:'Saturday',\
                        6:'Sunday',\
                        }, inplace=True)

    return df

In [ ]:
# Define function to add Month to data in one-hot-encoding form
def Month_one_hot_encoding(df):
    month = df.index.month
    df['Month'] = month
    df = df.join(pd.get_dummies(df['Month']))
    df.rename(columns={1:'Jan',\
                       2:'Feb',\
                       3:'Mar',\
                       4:'Apr',\
                        5:'May',\
                        6:'Jun',\
                        7:'Jul',\
                        8:'Aug',\
                        9:'Sep',\
                       10:'Oct',\
                       11:'Nov',\
                       12:'Dec'}                       
                        , inplace=True)
    return df

In [ ]:
# Define function to find monthly day-of-week average consumption for given appliance 
# in frequency of 1 second
def find_monthly_averge(df,ApplianceName):
    Average = np.zeros([7,13,86400])
    for i in range(7):
        for j in list(df.Month.unique()):
            avg = pd.DataFrame()
            values = df.loc[((df['DayOfWeek']==i) & (df['Month']==j))][ApplianceName].reset_index()[ApplianceName].values
            if len(values) == 86400*3:
                avg = pd.DataFrame({'firstweek':values[:86400], 'secondweek':values[86400:86400*2],\
                               'thirdweek':values[86400*2:86400*3]})
            if len(values) == 86400*4:
                avg = pd.DataFrame({'firstweek':values[:86400], 'secondweek':values[86400:86400*2],\
                               'thirdweek':values[86400*2:86400*3], 'fourthweek':values[86400*3:86400*4]})
            if len(values) == 86400*5:
                avg = pd.DataFrame({'firstweek':values[:86400], 'secondweek':values[86400:86400*2],\
                               'thirdweek':values[86400*2:86400*3], 'fourthweek':values[86400*3:86400*4],\
                               'fifthweek':values[86400*4:86400*5]})
            Average[i,j] = avg.mean(axis=1)
    return Average

In [ ]:
# Define function to assign monthly average value to a column in data
def assign_monthly_average(df,ApplianceName,Value):
    colname = ApplianceName+'MonthlyAvg'
    df[colname] = 0
    for i in range(7): 
        for j in list(df.Month.unique()): 
            mask = (df['DayOfWeek']==i) & (df['Month']==j)
            #n = (df[colname].loc[((df['DayOfWeek']==i) & (df['Month']==j))].shape[0] // 86400)
            n = (mask.sum())//86400
            l = list(Value[i,j])*n
            #df[colname].loc[((df['DayOfWeek']==i) & (df['Month']==j))] = l
            df[colname].loc[mask] =l
    return df

In [ ]:
# Define function to fill NaN value with average of the same day-of-week in the particular
# month for that timing 
def fill_missing_value(df,ApplianceName):
    colname = ApplianceName+'MonthlyAvg'
    df[ApplianceName].loc[df[ApplianceName].isnull()] = df[colname]
    return df

In [ ]:
# Define function to fill All NaN value in the dataframe with average of the same day-of-week 
# in the particular month for that timing 
def fill_All_missing_value(df,colnames):
    for colname in colnames:
        Average = find_monthly_averge(df,colname)
        assign_monthly_average(df,colname,Average)
        fill_missing_value(df,colname)
    return df

In [ ]:
# One-Hot-Encode DayOfWeek & Month
#house1 = DayOfWeek_one_hot_encoding(house1)
#house1 = Month_one_hot_encoding(house1)

In [ ]:
# Check Month & Day Of Week in data
#print(house1.Month.unique())
#print(house1.DayOfWeek.unique())

In [ ]:
# Get list containing all appliances
Appliances = house1.columns[:7].tolist()
Appliances

In [ ]:
house1_1s = fill_All_missing_value(house1,Appliances)

In [ ]:
house1_1s.head()

In [ ]:
house1_1s['Total'] = house1_1s.iloc[:,0:7].sum(axis = 1)
house1_appliances = house1_1s[Appliances]
house1_appliances['Total'] = house1_1s['Total']

In [ ]:
house1_appliances.head()

In [ ]:
# Resampling: transform into other frequencies
house1_30s = house1_appliances.resample('30S').sum() #30 seconds
house1_1m = house1_appliances.resample('60S').sum() #1 minute
house1_1h = house1_appliances.resample('1h').sum() #1 hour
house1_1d = house1_appliances.resample('1d').sum() #1 day

In [ ]:
# Export to csv file, Note the filepath
col = house1_30s.columns
# house1_1s.to_csv(path_or_buf='~/Desktop/data-x-data/ProcessedData/House1/House1_1s.csv', sep=',', na_rep='', float_format=None, columns=col, header=True, index=True)
# house1_30s.to_csv(path_or_buf='~/Desktop/data-x-data/ProcessedData/House1/House1_30s.csv', sep=',', na_rep='', float_format=None, columns=col, header=True, index=True)
# house1_1m.to_csv(path_or_buf='~/Desktop/data-x-data/ProcessedData/House1/House1_1m.csv', sep=',', na_rep='', float_format=None, columns=col, header=True, index=True)
house1_1h.to_csv(path_or_buf='~/Desktop/data-x-data/ProcessedData/House1/House1_1h.csv', sep=',', na_rep='', float_format=None, columns=col, header=True, index=True)
house1_1d.to_csv(path_or_buf='~/Desktop/data-x-data/ProcessedData/House1/House1_1d.csv', sep=',', na_rep='', float_format=None, columns=col, header=True, index=True)